In [1]:
import numpy as np
import pandas as pd
import os
import re
from datetime import date

In [2]:
import warnings    # to avoid warning during executions
warnings.filterwarnings("ignore")

In [11]:
import sys
import mariadb
from pandas.io import sql

In [4]:
try: con = mariadb.connect( 
    user="root", 
    password="password", 
    host="localhost", 
    port=3306, 
    database="data_dashboard" 
)

except mariadb.Error as ex: 
    print(f"An error occurred while connecting to MariaDB: {ex}") 
    sys.exit(1) 

cur = con.cursor()

In [6]:
file_name = file_name = '..\..\dataExport\\tbl_pbr_lookup.xlsx'
pbr_lookup = pd.read_excel(file_name,sheet_name='Sheet1')

In [7]:
pbr_lookup

,instance,unit,species,start_date,end_date,remark
0,1.0,FPC13,1.0,1.629468e+09,1.633939e+09,NaN
1,2.0,FPC13,1.0,1.634652e+09,1.644489e+09,NaN
2,3.0,FPC13,11.0,1.625227e+09,1.628158e+09,NaN
3,4.0,FPC14,2.0,1.614764e+09,1.618393e+09,NaN
4,5.0,FPC14,7.0,1.625486e+09,1.639564e+09,NaN
5,6.0,FPC14,11.0,1.618562e+09,1.625229e+09,NaN
6,7.0,FPC21,2.0,1.634915e+09,1.646645e+09,NaN
7,8.0,FPC21,9.0,1.623051e+09,1.626772e+09,NaN
8,9.0,FPC22,3.0,1.620216e+09,1.634632e+09,NaN
9,10.0,FPC23,7.0,1.616427e+09,1.626691e+09,NaN


In [13]:
import sqlalchemy

In [14]:
database_username = 'root'
database_password = 'password'
database_ip       = '127.0.0.1:3306'
database_name     = 'data_dashboard'
database_connection = sqlalchemy.create_engine('mariadb+mariadbconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [19]:
tbl_unit = pd.read_sql('SELECT unit_id, unit_name FROM tbl_unit', database_connection)

In [20]:
tbl_unit

,unit_id,unit_name
0,31,FFP_AUTFU
1,19,FFP_FFP
2,3,FFU
3,14,FPC11
4,34,FPC11_AUT11
5,1,FPC11_FPP11
6,22,FPC12
7,35,FPC12_AUT
8,29,FPC12_FPP
9,8,FPC13


In [21]:
pbr_lookup['unit']

0     FPC13
1     FPC13
2     FPC13
3     FPC14
4     FPC14
5     FPC14
6     FPC21
7     FPC21
8     FPC22
9     FPC23
10    FPC24
11    FPC24
Name: unit, dtype: object

In [26]:
#df['CODE'] = df.apply(lambda x: 0 if x['distance'] in a else x['CODE'], axis=1)

In [27]:
pbr_lookup['unit'] = pbr_lookup['unit'].map(tbl_unit.set_index('unit_name')['unit_id'])

In [28]:
pbr_lookup

,instance,unit,species,start_date,end_date,remark
0,1.0,8,1.0,1.629468e+09,1.633939e+09,NaN
1,2.0,8,1.0,1.634652e+09,1.644489e+09,NaN
2,3.0,8,11.0,1.625227e+09,1.628158e+09,NaN
3,4.0,2,2.0,1.614764e+09,1.618393e+09,NaN
4,5.0,2,7.0,1.625486e+09,1.639564e+09,NaN
5,6.0,2,11.0,1.618562e+09,1.625229e+09,NaN
6,7.0,6,2.0,1.634915e+09,1.646645e+09,NaN
7,8.0,6,9.0,1.623051e+09,1.626772e+09,NaN
8,9.0,4,3.0,1.620216e+09,1.634632e+09,NaN
9,10.0,7,7.0,1.616427e+09,1.626691e+09,NaN


In [29]:
pbr_lookup.to_sql(con=database_connection, name='tbl_pbr_lookup', if_exists='append', index=False)

-1